In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_80fdb1cbcfba4ffc89754c5419936eaf = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-KpsSgZ6YTByw0VvYt62SM6470OAy9_yGH0B4gTdKFzL',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_80fdb1cbcfba4ffc89754c5419936eaf.get_object(Bucket='catsanddogs-donotdelete-pr-uve7xoh7zeegdm',Key='Book1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)



In [2]:
df_data_1.head()

,Postcode,Borough,Neighbourhood
0,M9Z,Not assigned,Not assigned
1,M9Y,Not assigned,Not assigned
2,M9X,Not assigned,Not assigned
3,M9W,Etobicoke,Northwest
4,M9V,Etobicoke,Albion Gardens


In [10]:

body = client_80fdb1cbcfba4ffc89754c5419936eaf.get_object(Bucket='catsanddogs-donotdelete-pr-uve7xoh7zeegdm',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2 = df_data_2.rename(columns={"Postal Code": "Postcode"})
df_data_2.head()


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
result = pd.merge(df_data_1, df_data_2, on='Postcode')


In [13]:
result.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,M9V,Etobicoke,Albion Gardens,43.739416,-79.588437
2,M9V,Etobicoke,Beaumond Heights,43.739416,-79.588437
3,M9V,Etobicoke,Humbergate,43.739416,-79.588437
4,M9V,Etobicoke,Jamestown,43.739416,-79.588437
5,M9V,Etobicoke,Mount Olive,43.739416,-79.588437
6,M9V,Etobicoke,Silverstone,43.739416,-79.588437
7,M9V,Etobicoke,South Steeles,43.739416,-79.588437
8,M9V,Etobicoke,Thistletown,43.739416,-79.588437
9,M9R,Etobicoke,Kingsview Village,43.688905,-79.554724
